https://wiki.thebiogrid.org/doku.php/biogrid_tab_version_2.0
http://pages.cs.wisc.edu/~legault/writeup-776.pdf

In [266]:
import numpy as np
import pandas as pd
import networkx as nx
from collections import defaultdict 
import random

In [267]:
data = pd.read_table("BIOGRID-ORGANISM-Homo_sapiens-3.5.182.tab2.txt", low_memory = False)
# Same organisms as in paper
# data = pd.read_table("BIOGRID-ORGANISM-Saccharomyces_cerevisiae_S288c-3.5.182.tab2.txt", low_memory = False)
# data = pd.read_table("BIOGRID-ORGANISM-Escherichia_coli_K12_W3110-3.5.182.tab2.txt", low_memory = False)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [268]:
data.head()

,#BioGRID Interaction ID,Entrez Gene Interactor A,Entrez Gene Interactor B,BioGRID ID Interactor A,BioGRID ID Interactor B,Systematic Name Interactor A,Systematic Name Interactor B,Official Symbol Interactor A,Official Symbol Interactor B,Synonyms Interactor A,...,Pubmed ID,Organism Interactor A,Organism Interactor B,Throughput,Score,Modification,Phenotypes,Qualifications,Tags,Source Database
0,103,6416,2318,112315,108607,-,-,MAP2K4,FLNC,JNKK|JNKK1|MAPKK4|MEK4|MKK4|PRKMK4|SAPKK-1|SAP...,...,9006895,9606,9606,Low Throughput,-,-,-,-,-,BIOGRID
1,117,84665,88,124185,106603,-,-,MYPN,ACTN2,CMD1DD|CMH22|MYOP|RCM4,...,11309420,9606,9606,Low Throughput,-,-,-,-,-,BIOGRID
2,183,90,2339,106605,108625,-,-,ACVR1,FNTA,ACTRI|ACVR1A|ACVRLK2|ALK2|FOP|SKR1|TSRI,...,8599089,9606,9606,Low Throughput,-,-,-,-,-,BIOGRID
3,278,2624,5371,108894,111384,-,-,GATA2,PML,DCML|IMD21|MONOMAC|NFE1B,...,10938104,9606,9606,Low Throughput,-,-,-,-,-,BIOGRID
4,418,6118,6774,112038,112651,RP4-547C9.3,-,RPA2,STAT3,REPA2|RP-A p32|RP-A p34|RPA32,...,10875894,9606,9606,Low Throughput,-,-,-,-,-,BIOGRID


In [269]:
# Extract interactions
val = data.copy()
val.set_index("Experimental System Type", inplace = True)
val = val.filter(items = ["Official Symbol Interactor A", "Official Symbol Interactor B", "Pubmed ID"])
# val = val.filter(like = "physical", axis = 0)
val = val.filter(like = "genetic", axis = 0)
val.head()

,Official Symbol Interactor A,Official Symbol Interactor B,Pubmed ID
Experimental System Type,,,
genetic,SLT2,MAPK7,16950928
genetic,BCR,HOXA9,12393433
genetic,ATM,TP53,9135004
genetic,NCOR1,AR,15598662
genetic,CTNNB1,CREBBP,15782138


In [270]:
# Create graph of all interactions
graph = nx.from_pandas_edgelist(val, "Official Symbol Interactor A", "Official Symbol Interactor B", create_using=nx.DiGraph)

In [271]:
# nx.draw_networkx(graph)

In [272]:
# Discover all subgraphs of graph
subgraphs = list(nx.weakly_connected_component_subgraphs(graph))

In [273]:
subgraphs_n = defaultdict(list)

In [274]:
# Categorize subgraphs depending on number of nodes
for subgraph in subgraphs:
    subgraphs_n[nx.number_of_nodes(subgraph)].append(subgraph)

In [275]:
subgraphs_n

defaultdict(list,
            {3528: [<networkx.classes.digraph.DiGraph at 0x1a278d8ba8>],
             2: [<networkx.classes.digraph.DiGraph at 0x1a21d0d518>,
             4: [<networkx.classes.digraph.DiGraph at 0x1a278d8da0>,
             3: [<networkx.classes.digraph.DiGraph at 0x1a53632f60>,
             7: [<networkx.classes.digraph.DiGraph at 0x1a5365f3c8>],
             1: [<networkx.classes.digraph.DiGraph at 0x1a5365f080>,
             6: [<networkx.classes.digraph.DiGraph at 0x1a28d88f28>]})

In [276]:
# Implementation of distance function
a = pd.DataFrame(np.array([[0,1,1], [0,0,0], [0,0,0]]))
b = pd.DataFrame(np.array([[0,0,0], [1,0,1], [0,0,0]]))
c = pd.DataFrame(np.array([[0,0,0], [0,0,0], [1,1,0]]))
d = pd.DataFrame(np.array([[0,0,0], [1,0,0], [1,0,0]]))
e = pd.DataFrame(np.array([[0,0,0], [1,0,0], [0,1,0]]))

def attributes(x):
    """
    Returns [number of source nodes, number of outgoing edges per node...]
    which is equivalent to d(x, 0) for connectivity matrix x
    
    x is a pandas dataframe
    """
    n = len(x)
    
    # Initialize with counter of source nodes
    result = [0]
    
    # Increment counter for number of source nodes
    for j in range(n):
        for i in range(n):
            if x.iloc[i,j] != 0:
                break
            elif i == n - 1:
                result[0] += 1
    result[0] = str(result[0])
    # Append sorted list of number of outgoing edges per node
    result.extend(sorted([str(int(sum(x.iloc[i]))) for i in range(n)]))
    result_string = "".join(result)
    return result_string

# Testing connectivity matrices in paper
print(attributes(a))
print(attributes(b))
print(attributes(c))
print(attributes(d))
print(attributes(e))

1002
1002
1002
2011
1011


In [277]:
# for n in subgraphs_n.keys():
for subgraph in subgraphs_n[3]:
    print(attributes(nx.to_pandas_adjacency(subgraph)))

2011
1011
1011
1002
1002
1012


In [278]:
# Hash function implementation
def subgraph_hash(n, r):
    """
    n is size of subgraph
    r is measurement of similarity (in paper)
    """
    random.seed(24)
    positions = sorted(random.sample(range(n + 1), n + 1 - r))
#     print(positions)
    table = defaultdict(list)
    for subgraph in subgraphs_n[n]:
        characteristics = attributes(nx.to_pandas_adjacency(subgraph))
        hash_key = []
        for p in positions:
            hash_key.append(characteristics[p])
        hash_key_string = "".join(hash_key)
        table[hash_key_string].append(subgraph)
    print(table)
subgraph_hash(3, 1)

defaultdict(<class 'list'>, {'211': [<networkx.classes.digraph.DiGraph object at 0x1a53632f60>], '111': [<networkx.classes.digraph.DiGraph object at 0x1a53632f28>, <networkx.classes.digraph.DiGraph object at 0x1a5365fc88>], '102': [<networkx.classes.digraph.DiGraph object at 0x1a5365fbe0>, <networkx.classes.digraph.DiGraph object at 0x1a28d88f60>], '112': [<networkx.classes.digraph.DiGraph object at 0x1a2610e9b0>]})


In [279]:
# We need a good organism to look at that has a large number of small subgraphs (as in the paper (not sure why we don't have the same thing))
# The distance function is something the person randomly came up with very simple reasoning
# We can come up with different distance functions (pretty easily after understanding the distance function in the paper) and draw nice graphs of their respective accuracies of identifying similar subgraphs?
# Our paper is basically a direct extension of the paper we read